In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
from sklearn.metrics import plot_confusion_matrix

In [ ]:
data = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
data.describe()
data.info()

In [ ]:
data.isna().sum()
#changing the 0s to the mean value of the values in the columns to fix missing data issue
X= data.replace(0, np.NaN)
X= data.fillna(data.mean(), inplace=True)



In [ ]:
# Looking at the data there are no null values but there are 0s where there should be actual values such as in insulin
#first lets split the data
X= data.drop(['Outcome'], axis=1)
X= X.drop([0], axis=0)
y= data['Outcome']
y= y.drop([0], axis=0)
print(X.shape)
print(y.shape)





In [ ]:
#splitting data into training and test sets 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.33, random_state=42) #scaling the data 
scaler = MinMaxScaler(feature_range=(0, 1)) 
rescaledX_train = scaler.fit_transform(X_train) 
rescaledX_test = scaler.transform(X_test) 
#creating knn classifier 
knn = KNeighborsClassifier(algorithm= 'brute') 
params= {'n_neighbors':[4,5,6,7], 'leaf_size':[1,3,5], 'n_jobs':[-1]}
diabetic1= GridSearchCV(knn, params,cv=5)


In [ ]:
diabetic_cv= diabetic1.fit(X_train, y_train)
y_pred= diabetic1.predict(X_test)
score = diabetic1.score(X_test, y_test)

print("Accuracy: {}".format(score))
print("Tuned Model Parameters: {}".format(diabetic1.best_params_))

In [ ]:
#testing with rescaled data
diabetic_cv2= diabetic1.fit(rescaledX_train, y_train)
y_pred2= diabetic1.predict(rescaledX_test)
score2 = diabetic1.score(rescaledX_test, y_test)

print("Accuracy: {}".format(score))
print("Tuned Model Parameters: {}".format(diabetic1.best_params_))

In [ ]:
#We observed that both the scaled and unscaled version of the model gives a 75.98% accuracy score hence using either version will give similar results
#plotting the confusion matrix
plot_confusion_matrix(diabetic_cv2, rescaledX_test, y_test)  
plt.show()  
#The confusion matrix shows that the model performs fairly well with decent recall and percision with a large amount of true positives and true negatives

In [ ]:
#Though the model performs fairly well there is still room for improvement let's test if another classifiaction algorithm fits the data better

logreg= LogisticRegression()
#parameters for the model
c_space = np.logspace(-5, 8, 15)
params_log = {'C': c_space}
#tuning the model
logreg_cv= GridSearchCV(logreg, params_log,cv=5)

In [ ]:
diabetic3= logreg_cv.fit(rescaledX_train, y_train)
y_pred3= logreg_cv.predict(rescaledX_test)
R2 = logreg_cv.score(rescaledX_test, y_test)

print("Accuracy: {}".format(R2))
print("Tuned Model Parameters: {}".format(diabetic3.best_params_))

In [ ]:
#the Logistc regression model has a 79% accuraccy score meaning it fits the data better than the knn model
plot_confusion_matrix(diabetic3, rescaledX_test, y_test)  
plt.show() 
#The confusion matrix shows that the logreg model has higher percision but lower recall which is actually better for this type of data

In [ ]:
#best model for the data

best_tuned= LogisticRegression(C= 268.2695795279727)
best_tuned.fit(rescaledX_train, y_train)
best_pred= best_tuned.predict(rescaledX_test)
print('the predictions are {}'.format(best_pred))
print('the actual results are {}'.format(y_test))

In [ ]:
#the predictions are fairly accurate
predictions = pd.DataFrame( best_pred).value_counts()
answers= pd.DataFrame(y_test).value_counts()
#comparing predictions and actual answers with a bar graph
predictions.plot(kind= 'bar', color='b', xlabel='outcome', title= 'predictions')
plt.grid(linewidth=1)
plt.show()
answers.plot(kind='bar', color='g', xlabel= 'outcome', title= 'True outcome')
plt.grid(linewidth=1)
#The graphs show that there are some wrong predictions but not a large amount of them, which coincides with the testing of the model
#in conclusion this model is fairly accurate at predicting whether a patient has diabetes or not given the data provided